CS524: Introduction to Optimization Lecture 20
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 25, 2019
--------------

In [1]:
%load_ext gams.magic
m = gams.exchange_container

### Either-Or constraints and semiint variables

Dorian Auto is considering manufacturing three types of autos: 
compact, midsize and large.  The resources required for, and the 
profits yielded by, each type of car are shown below.
At present 6,000 tons of steel and 60,000 hours of labor are avaiable.
In order for production of a type of car to be economically feasible, at 
least 1,000 cars of that type must be produced.  
Formulate an IP to maximize Dorian's profit.

In [2]:
%%gams
set type /compact, midsize, tank/;
set ingredients /labor  "in hours"
               steel  "in tons"/;
table resources(ingredients,type)
      compact   midsize   tank
steel  1.5        3        5
labor  30         25       40;

parameter profit(type) / compact 2, midsize 3, tank 4/;

parameter available(ingredients) / steel 6000, labor 60000/;

* minimum amount of a car to be produced, for viability
scalar carsRequired / 1000 / 
       bigM         / 3000 /;

Now solve the model

In [3]:
%%gams
integer variables cars(type)   "how many cars of this type";
cars.lo(type)=0;
cars.up(type)=bigM;
variable totalProfit;
binary variable produce(type)  "produce this type of car";

equations
        EQeither(type)   "either constraint"
        EQor(type)       "or constraint"
        resourceConstraint
        objective;

EQeither(type)..
        cars(type) =l= bigM*produce(type);
EQor(type)..
        carsRequired - cars(type) =l= bigM*(1-produce(type));

resourceConstraint(ingredients)..
        sum(type,resources(ingredients,type)*cars(type)) =l= 
                 available(ingredients);

objective..
        totalProfit =e= sum(type,profit(type)*cars(type));

model dorian/EQeither, EQor, resourceConstraint, objective/;

option limrow=0, limcol=0;
dorian.optcr=0;

solve dorian using mip maximizing totalProfit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),6000.0,9,7,MIP,CPLEX,0.013


In [4]:
cars = m.data['cars'].records
totalProfit = m.data['totalProfit'].toValue()
print('Total Profit is {:5.1f}'.format(totalProfit))
print('cars built:')
for t in range(len(cars.index)):
    print('{0:>8s} {1:8.0f}'.format(cars['type'][t],cars['level'][t]))

Total Profit is 6000.0
cars built:
 compact        0
 midsize     2000
    tank        0


In [11]:
# Alternative form of output
cars = m.data['cars'].toList()
print('Total Profit is {:5.1f}'.format(totalProfit))
print('cars built:')
for x in cars:
    print('{0:>8s} {1:8.0f}'.format(x[0],x[1]))

Total Profit is 6000.0
cars built:
 compact        0
 midsize     2000
    tank        0


Now solve the model with semiint variables

In [12]:
%%gams
* Note that semicont variables also exist!
semiint variables carsI(type)   "how many cars of this type";
carsI.lo(type)=carsRequired;
carsI.up(type)=bigM;

resourceConstraint(ingredients)..
        sum(type,resources(ingredients,type)*carsI(type)) =l= 
                 available(ingredients);

objective..
        totalProfit =e= sum(type,profit(type)*carsI(type));

model dorianI /resourceConstraint, objective/;

option limrow=0, limcol=0;
dorianI.optcr=0;

solve dorianI using mip maximizing totalProfit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),6000.0,3,4,MIP,CPLEX,0.013


In [13]:
carsI = m.data['carsI'].toList()
totalProfit = m.data['totalProfit'].toValue()
print('Total Profit is {:5.1f}'.format(totalProfit))
print('cars built:')
for x in carsI:
  print('{0:>8s} {1:8.0f}'.format(x[0],x[1]))

Total Profit is 6000.0
cars built:
 compact        0
 midsize     2000
    tank        0


In [14]:
%gams_cleanup --closedown